<a href="https://colab.research.google.com/github/Bishara10/ML-Classification-model-For-EEG-Recordings/blob/main/EEG_Classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/BPy5/ML-Classification-model-For-EEG-Recordings.git

Cloning into 'ML-Classification-model-For-EEG-Recordings'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 295 (delta 16), reused 2 (delta 0), pack-reused 256
Receiving objects: 100% (295/295), 72.68 MiB | 21.50 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Updating files: 100% (246/246), done.


In [5]:
file_names_adhd_dataset = ['v8p.csv', 'v35p.csv', 'v37p.csv', 'v36p.csv', 'v38p.csv', 'v6p.csv', 'v40p.csv', 'v39p.csv', 'v3p.csv', 'v29p.csv', 'v30p.csv', 'v31p.csv', 'v28p.csv', 'v279.csv', 'v33p.csv', 'v25p.csv', 'v286.csv', 'v250.csv', 'v24p.csv', 'v246.csv', 'v263.csv', 'v27p.csv', 'v254.csv', 'v284.csv', 'v238.csv', 'v265.csv', 'v227.csv', 'v215.csv', 'v236.csv', 'v22p.csv', 'v21p.csv', 'v231.csv', 'v244.csv', 'v213.csv', 'v274.csv', 'v200.csv', 'v206.csv', 'v19p.csv', 'v173.csv', 'v270.csv', 'v234.csv', 'v179.csv', 'v181.csv', 'v18p.csv', 'v14p.csv', 'v196.csv', 'v209.csv', 'v183.csv', 'v12p.csv', 'v219.csv', 'v15p.csv', 'v20p.csv', 'v198.csv', 'v190.csv', 'v1p.csv', 'v32p.csv', 'v177.csv', 'v10p.csv', 'v34p.csv', 'v288.csv', 'v204.csv']
# print(len(file_names_adhd_dataset))
file_names_control_dataset = ['v51p.csv', 'v59p.csv', 'v304.csv', 'v45p.csv', 'v54p.csv', 'v58p.csv', 'v53p.csv', 'v60p.csv', 'v56p.csv', 'v46p.csv', 'v308.csv', 'v42p.csv', 'v43p.csv', 'v306.csv', 'v48p.csv', 'v50p.csv', 'v52p.csv', 'v302.csv', 'v44p.csv', 'v47p.csv', 'v307.csv', 'v57p.csv', 'v303.csv', 'v127.csv', 'v134.csv', 'v121.csv', 'v299.csv', 'v151.csv', 'v116.csv', 'v125.csv', 'v133.csv', 'v300.csv', 'v147.csv', 'v298.csv', 'v113.csv', 'v118.csv', 'v117.csv', 'v120.csv', 'v111.csv', 'v107.csv', 'v114.csv', 'v108.csv', 'v115.csv', 'v305.csv', 'v140.csv', 'v149.csv', 'v309.csv', 'v123.csv', 'v138.csv', 'v112.csv', 'v41p.csv', 'v129.csv', 'v143.csv', 'v310.csv', 'v131.csv', 'v297.csv', 'v109.csv', 'v49p.csv', 'v55p.csv', 'v110.csv']
# print(len(file_names_control_dataset))

pathControl = '/content/ML-Classification-model-For-EEG-Recordings/Dataset/CSV/controlcsv/'
pathADHD = '/content/ML-Classification-model-For-EEG-Recordings/Dataset/CSV/adhdcsv/'

outputPath = '/content/preprocessed_dataset/'

In [29]:
import numpy as np
import pandas as pd
from scipy.signal import resample, spectrogram, butter, filtfilt
import scipy.signal

# Parameters
original_fs = 128  # Original sampling frequency in Hz
target_fs = 64  # Target sampling frequency in Hz
lowcut = 10.0  # Lower bound of the bandpass filter
highcut = 40.0  # Upper bound of the bandpass filter
nperseg = 128  # Number of samples per segment
noverlap = 64  # Number of overlapping samples

# Placeholder for processed data
dataADHD = []
dataControl = []

# Function to downsample the signal
def downsample_signal(signal, original_fs, target_fs):
    num_samples = int(len(signal) * target_fs / original_fs)
    print(f"Original signal length: {len(signal)}, Downsampled signal length: {num_samples}")
    return resample(signal, num_samples)

# Function to apply the bandpass filter
def bandPassFilter(signal, fs):
    lowcut = 10.0
    highcut = 40.0
    nyq = 0.5 * 4000
    low = lowcut / nyq
    high = highcut / nyq
    order = 2
    b, a = scipy.signal.butter(order, [low, high], 'bandpass', analog=False)
    print(f"Applying bandpass filter: lowcut={lowcut} Hz, highcut={highcut} Hz, fs={fs} Hz")
    print(f"Normalized frequencies: low={low}, high={high}")
    y = scipy.signal.filtfilt(b, a, signal, axis=0)
    return y

# Function to pad spectrograms to a consistent length
def pad_spectrogram(sxx, max_length):
    if sxx.shape[1] < max_length:
        padding = np.zeros((sxx.shape[0], max_length - sxx.shape[1]))
        sxx = np.hstack((sxx, padding))
    else:
        sxx = sxx[:, :max_length]
    return sxx

# Preprocessing function
def preprocess_file(input_file, group_category, max_time_bins):
    df = pd.read_csv(input_file)
    spectrograms = []

    for column in df.columns:
        signal = df[column].values
        downsampled_signal = downsample_signal(signal, original_fs, target_fs)
        filtered_signal = bandPassFilter(downsampled_signal, target_fs)

        f, t, Sxx = spectrogram(filtered_signal, fs=target_fs, nperseg=nperseg, noverlap=noverlap)
        Sxx = pad_spectrogram(Sxx, max_time_bins)
        spectrograms.append(Sxx)

    spectrogram_stack = np.stack(spectrograms, axis=-1)
    print(f"Spectrogram stack shape: {spectrogram_stack.shape}")

    if group_category == 'ADHD':
        dataADHD.append(spectrogram_stack)
    elif group_category == 'CONTROL':
        dataControl.append(spectrogram_stack)

# Example usage to determine max time bins
# Calculate the maximum time bins across all files
def calculate_max_time_bins(files):
    max_time_bins = 0
    for file in files:
        df = pd.read_csv(file)
        for column in df.columns:
            signal = df[column].values
            downsampled_signal = downsample_signal(signal, original_fs, target_fs)
            f, t, Sxx = spectrogram(downsampled_signal, fs=target_fs, nperseg=nperseg, noverlap=noverlap)
            max_time_bins = max(max_time_bins, Sxx.shape[1])
    return max_time_bins



In [27]:
# Preprocess all recordings for a specific group
def preprocess_group(group_path, group_file_names, output_path, group_category, max_time_bins):
  for filename in group_file_names:
    file_name_and_path = group_path + filename;
    preprocess_file(file_name_and_path, group_category, max_time_bins)
  print("=============================== Preprocessing group successful =============================== ")

In [ ]:
filesADHD = ['/content/ML-Classification-model-For-EEG-Recordings/Dataset/CSV/adhdcsv/' + x for x in file_names_adhd_dataset]
filesControl = ['/content/ML-Classification-model-For-EEG-Recordings/Dataset/CSV/controlcsv/' + x for x in file_names_control_dataset]
x = calculate_max_time_bins(filesADHD)
y = calculate_max_time_bins(filesControl)
maxTimeBins = max(x, y)

In [30]:
preprocess_group(pathADHD, file_names_adhd_dataset, outputPath + '/ADHD/', 'ADHD', maxTimeBins)
preprocess_group(pathControl, file_names_control_dataset, outputPath + '/Control/', 'CONTROL', maxTimeBins)

Streaming output truncated to the last 5000 lines.
Original signal length: 17323, Downsampled signal length: 8661
Applying bandpass filter: lowcut=10.0 Hz, highcut=40.0 Hz, fs=64 Hz
Normalized frequencies: low=0.005, high=0.02
Original signal length: 17323, Downsampled signal length: 8661
Applying bandpass filter: lowcut=10.0 Hz, highcut=40.0 Hz, fs=64 Hz
Normalized frequencies: low=0.005, high=0.02
Original signal length: 17323, Downsampled signal length: 8661
Applying bandpass filter: lowcut=10.0 Hz, highcut=40.0 Hz, fs=64 Hz
Normalized frequencies: low=0.005, high=0.02
Spectrogram stack shape: (65, 336, 19)
Original signal length: 12739, Downsampled signal length: 6369
Applying bandpass filter: lowcut=10.0 Hz, highcut=40.0 Hz, fs=64 Hz
Normalized frequencies: low=0.005, high=0.02
Original signal length: 12739, Downsampled signal length: 6369
Applying bandpass filter: lowcut=10.0 Hz, highcut=40.0 Hz, fs=64 Hz
Normalized frequencies: low=0.005, high=0.02
Original signal length: 12739,

In [32]:
dataADHD

[array([[[7.43457787e+02, 2.09953661e+03, 4.36457276e+01, ...,
          2.80843182e+01, 1.23001478e+02, 2.00753248e+01],
         [3.34062653e+02, 5.87357192e+02, 1.76606945e+00, ...,
          7.03452203e+00, 3.72155988e+01, 5.64448625e+00],
         [2.22951848e+02, 2.38163501e+02, 4.95656117e+01, ...,
          3.97697155e+01, 1.55205014e+00, 2.04532579e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
        [[3.19616763e+05, 2.93035783e+05, 9.84417731e+03, ...,
          1.53785228e+04, 6.00916520e+03, 7.66521265e+02],
         [1.12442446e+03, 5.51969641e+03, 1.16808030e+02, ...,
          2.43198581e+02, 4.53090948e+02, 7.43954045e+02],
       

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense

# Define CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(65, None, 19)))  # None for variable time dimension
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())  # Global pooling to handle variable length
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [33]:
# Convert lists to numpy arrays
dataADHD = np.array(dataADHD)
dataControl = np.array(dataControl)

# Create labels
labelsADHD = np.ones(len(dataADHD))
labelsControl = np.zeros(len(dataControl))

# Combine data and labels
X = np.concatenate((dataADHD, dataControl), axis=0)
y = np.concatenate((labelsADHD, labelsControl), axis=0)

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
model.evaluate(X, y)


Epoch 1/10
3/3 [==============================] - 7s 2s/step - loss: 34.6190 - accuracy: 0.3438 - val_loss: 0.0331 - val_accuracy: 0.9600
Epoch 2/10
3/3 [==============================] - 6s 2s/step - loss: 7.5240 - accuracy: 0.5729 - val_loss: 29.9913 - val_accuracy: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 4s 2s/step - loss: 8.8761 - accuracy: 0.6146 - val_loss: 0.3301 - val_accuracy: 0.8800
Epoch 4/10
3/3 [==============================] - 5s 2s/step - loss: 6.5390 - accuracy: 0.4375 - val_loss: 2.3985 - val_accuracy: 0.7200
Epoch 5/10
3/3 [==============================] - 5s 1s/step - loss: 3.0829 - accuracy: 0.6979 - val_loss: 16.4091 - val_accuracy: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 4s 2s/step - loss: 3.0385 - accuracy: 0.6771 - val_loss: 4.4310 - val_accuracy: 0.5200
Epoch 7/10
3/3 [==============================] - 5s 2s/step - loss: 1.8440 - accuracy: 0.5729 - val_loss: 3.4675 - val_accuracy: 0.4800
Epoch 8/10
3/3 [==============

[1.68556809425354, 0.6115702390670776]